# Data Preprocessing

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
libdir = os.chdir(r'C:\Users\Nidhi\Desktop\Saransh\FraudDetection')

In [4]:
cols = range(1,12)
file = r'./data/interim/cleansed.csv'
df = pd.read_csv(file,usecols = cols)
df.head(10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.0,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.0,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.0,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.0,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.0,0.00,0,0
5,1,PAYMENT,7817.71,C90045638,53860.00,46042.29,M573487274,0.0,0.00,0,0
6,1,PAYMENT,7107.77,C154988899,183195.00,176087.23,M408069119,0.0,0.00,0,0
7,1,PAYMENT,7861.64,C1912850431,176087.23,168225.59,M633326333,0.0,0.00,0,0
8,1,PAYMENT,4024.36,C1265012928,2671.00,0.00,M1176932104,0.0,0.00,0,0
9,1,DEBIT,5337.77,C712410124,41720.00,36382.23,C195600860,41898.0,40348.79,0,0


### Feature Engineering

In [5]:
def retCustType(id):
    if id[0]=='C':
        return 'Customer'
    elif id[0]=='M':
        return 'Merchant'
    else:
        return 'NA'

def retAmntEqual(row):
    if np.abs(row.oldbalanceOrg - row.newbalanceOrig) - row.amount <=.99 or np.abs(row.oldbalanceOrg - row.newbalanceOrig) - row.amount >=-.99:
        return 1
    else:
        return 0
    return row

def getDays(row):
    days = (row.step/24)+1 
    return int(round(days,0))

def getHours(row):
    hours = row.step%24
    return hours

def retOneHot(dataFrame):
    return pd.get_dummies(dataFrame) 

Extracting more information from features like the type of Customer who originated/received amount, whether the transferred amount reflects correctly in the sender's old and new balances. Extracting hours and days from the step column, etc.

In [6]:
df['CustTypeOrig'] = df['nameOrig'].apply(retCustType)
df['CustTypeDest'] = df['nameDest'].apply(retCustType)

In [7]:
df['CustTypeDest'].value_counts()

Merchant    56
Customer    44
Name: CustTypeDest, dtype: int64

In [8]:
df['transferAmntCheck'] = df.apply(retAmntEqual, axis =1)

In [9]:
df['Day'] = df.apply(getDays,axis =1)
df['Hour'] = df.apply(getHours,axis =1)

In [10]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,CustTypeOrig,CustTypeDest,transferAmntCheck,Day,Hour
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,Customer,Merchant,1,1,1
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,Customer,Merchant,1,1,1
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,Customer,Customer,1,1,1
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,Customer,Customer,1,1,1
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,Customer,Merchant,1,1,1


In [11]:
df.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud', 'CustTypeOrig', 'CustTypeDest', 'transferAmntCheck',
       'Day', 'Hour'],
      dtype='object')

In [12]:
cols = ['type','isFlaggedFraud', 'CustTypeOrig', 'CustTypeDest', 'Day', 'Hour','transferAmntCheck']

for col in cols:
    oneHotDf = pd.get_dummies(df[col],prefix = col)
    df = pd.concat([df,oneHotDf],axis = 1)

In [13]:
df.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud', 'CustTypeOrig', 'CustTypeDest', 'transferAmntCheck',
       'Day', 'Hour', 'type_CASH_OUT', 'type_DEBIT', 'type_PAYMENT',
       'type_TRANSFER', 'isFlaggedFraud_0', 'CustTypeOrig_Customer',
       'CustTypeDest_Customer', 'CustTypeDest_Merchant', 'Day_1', 'Hour_1',
       'transferAmntCheck_1'],
      dtype='object')

In [14]:
df.shape

(100, 27)

In [15]:
df.head(10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,...,type_DEBIT,type_PAYMENT,type_TRANSFER,isFlaggedFraud_0,CustTypeOrig_Customer,CustTypeDest_Customer,CustTypeDest_Merchant,Day_1,Hour_1,transferAmntCheck_1
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.0,0.00,0,...,0,1,0,1,1,0,1,1,1,1
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.0,0.00,0,...,0,1,0,1,1,0,1,1,1,1
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.0,0.00,1,...,0,0,1,1,1,1,0,1,1,1
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.0,0.00,1,...,0,0,0,1,1,1,0,1,1,1
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.0,0.00,0,...,0,1,0,1,1,0,1,1,1,1
5,1,PAYMENT,7817.71,C90045638,53860.00,46042.29,M573487274,0.0,0.00,0,...,0,1,0,1,1,0,1,1,1,1
6,1,PAYMENT,7107.77,C154988899,183195.00,176087.23,M408069119,0.0,0.00,0,...,0,1,0,1,1,0,1,1,1,1
7,1,PAYMENT,7861.64,C1912850431,176087.23,168225.59,M633326333,0.0,0.00,0,...,0,1,0,1,1,0,1,1,1,1
8,1,PAYMENT,4024.36,C1265012928,2671.00,0.00,M1176932104,0.0,0.00,0,...,0,1,0,1,1,0,1,1,1,1
9,1,DEBIT,5337.77,C712410124,41720.00,36382.23,C195600860,41898.0,40348.79,0,...,1,0,0,1,1,1,0,1,1,1


In [16]:
drop_cols = ['step','type','Day','Hour','nameOrig','nameDest','CustTypeOrig','CustTypeDest','isFlaggedFraud','transferAmntCheck']
df.drop(drop_cols,axis = 1,inplace = True)

In [20]:
with pd.ExcelWriter('./data/interim/preprocessed.xlsx') as writer:
    df.to_excel(writer,index = False)